In [1]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime
from itertools import product
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
import pandas as pd
import random
import time

# 回测框架
import backtrader as bt

# 基础函数
import utilsJ

# 策略

In [2]:
class BW_comp(bt.Strategy):
    
    params = (
        # General params
        ('printlog', False),
        ('units', 300),
        ('p_stake', 100),

        # Indicator params
        ('bollinger_per', 20),
        ('bollinger_dev', 2),
        ('wave_period_buy', 7),
        ('wave_period_sell', 7),

        # ATR params
        ('using_atr', False),
        ('atr_period', 14),
        ('atr_percent', 1),
    )

    
    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))

    
    def __init__(self):

        # Global initialization
        self.atr_initial = self.broker.get_cash()
        self.inds = dict()
        self.profit_i = dict()
        self.cash_unit = self.broker.get_cash() / self.params.units

        for d in self.datas:

            self.inds[d] = dict()
            self.profit_i[d._name] = 0

            # Keep references to lines in the data[0] dataseries
            self.inds[d]['dataclose'] = d.close
            self.inds[d]['datahigh'] = d.high
            self.inds[d]['datalow'] = d.low

            # Keep references to executed order & price
            self.inds[d]['buyprice'] = 0
            self.inds[d]['sellprice'] = 0
            self.inds[d]['order'] = None

            # Add indicators
            ## Bollinger Bands indicator
            self.inds[d]['bollinger'] = bt.indicators.BollingerBands(d, 
                                                        period=self.params.bollinger_per, 
                                                        devfactor=self.params.bollinger_dev)
            self.inds[d]['bollinger_buy'] = bt.And(self.inds[d]['dataclose'](0) > self.inds[d]['bollinger'].mid(0), 
                                                   self.inds[d]['dataclose'](-1) < self.inds[d]['bollinger'].mid(-1))

            self.inds[d]['bollinger_sell'] = bt.And(self.inds[d]['dataclose'](0) < self.inds[d]['bollinger'].top(0), 
                                                    self.inds[d]['dataclose'](-1) > self.inds[d]['bollinger'].top(-1))


            ## MovingAverageSimple indicator
            self.inds[d]['sma_buy'] = bt.indicators.SimpleMovingAverage(d, period=self.params.wave_period_buy)
            self.inds[d]['sma_sell'] = bt.indicators.SimpleMovingAverage(d, period=self.params.wave_period_sell)
            
            ## Wave indicator
            self.inds[d]['wave_buy'] = bt.And(self.inds[d]['sma_buy'](0) > self.inds[d]['sma_buy'](-1), 
                                              self.inds[d]['sma_buy'](-1) < self.inds[d]['sma_buy'](-2), 
                                              self.inds[d]['sma_buy'](-2) < self.inds[d]['sma_buy'](-3),
                                              self.inds[d]['sma_buy'](0) > self.inds[d]['bollinger'].mid(0))
                                             

            self.inds[d]['wave_sell'] = bt.And(self.inds[d]['sma_sell'](0) < self.inds[d]['sma_sell'](-1), 
                                               self.inds[d]['sma_sell'](-1) > self.inds[d]['sma_sell'](-2), 
                                               self.inds[d]['sma_sell'](-2) > self.inds[d]['sma_sell'](-3))
        
            ## ATR indicator
            self.inds[d]['tr'] = bt.indicators.Max((self.inds[d]['datahigh'](0) - self.inds[d]['datalow'](0)), 
                                                    abs(self.inds[d]['dataclose'](-1) - self.inds[d]['datahigh'](0)),
                                                    abs(self.inds[d]['dataclose'](-1) - self.inds[d]['datalow'](0)))
            self.inds[d]['atr'] = bt.indicators.SimpleMovingAverage(self.inds[d]['tr'], period=self.params.atr_period)


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                if self.inds[self.data_tracker]['wave_buy']:
                    self.inds[self.data_tracker]['buyprice']
                self.log('买单执行, 代码：%s, 价格:%.2f, 股数:%i, 持有现金:%i' %
                         (order.info['name'],
                          order.executed.price,
                          order.executed.size,
                          self.broker.get_cash()))
                self.profit_i[order.info['name']] -= order.executed.price*order.executed.size

            else:  # Sell
                self.inds[self.data_tracker]['sellprice'] = order.executed.price
                self.log('卖单执行, 代码：%s, 价格:%.2f, 股数:%i, 持有现金:%i' %
                        (order.info['name'],
                         order.executed.price,
                         -order.executed.size,
                         self.broker.get_cash()))
                self.profit_i[order.info['name']] -= order.executed.price*order.executed.size

        elif order.status in [order.Canceled]:
            self.log('订单取消：被撤销')

        elif order.status in [order.Margin]:
            self.log('订单取消：资金不足，代码：%s，持有资金:%i' %
                        (order.info['name'],
                         self.broker.get_cash()))

        elif order.status in [order.Rejected]:
            self.log('订单取消：被拒绝')

        # Write down: no pending order
        #self.order = None


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('交易收益：代码：%s, 毛利润 %.2f, 净利润 %.2f' %
                 (trade.history[0]['event']['order'].info['name'], 
                  trade.pnl, trade.pnlcomm))
        

    def next(self):

        buy_list = []
        sell_list = []

        for d in self.datas:
        # Check if an order is pending ... if yes, we cannot send a 2nd one
            if self.inds[d]['order']:
                return

            buy_indicator = self.inds[d]['wave_buy']
            sell_indicator = self.inds[d]['wave_sell'] or self.inds[d]['bollinger_sell']

            if sell_indicator and self.getposition(d).size > 0:
                sell_list.append((d, -1))

            elif buy_indicator and self.getposition(d).size == 0:
                if self.params.using_atr:
                    opt_pos = np.floor(self.atr_initial * self.params.atr_percent / self.params.p_stake / self.inds[d]['atr'][0])
                    if opt_pos*self.params.p_stake > 0 and self.getposition(d).size < opt_pos*self.params.p_stake:
                        buy_list.append((d, 
                                        opt_pos*self.params.p_stake-self.getposition(d).size, 
                                        (opt_pos*self.params.p_stake-self.getposition(d).size)*self.inds[d]['dataclose'][0]))
                else:
                    lots = np.floor(self.cash_unit / (self.params.p_stake*self.inds[d]['dataclose'][0]))*self.params.p_stake
                    buy_list.append((d, lots, lots * self.inds[d]['dataclose'][0]))
        
        for s_order in sell_list:
            self.data_tracker = s_order[0]
            self.log('卖单创建, 代码: %s, 价格: %.2f, 股数:%i, 现有持仓: %i' % 
                        (s_order[0]._name, 
                        self.inds[s_order[0]]['dataclose'][0],
                        s_order[1] if s_order[1] != -1 else self.getposition(s_order[0]).size,
                        self.getposition(s_order[0]).size))
            self.order = self.sell(data=s_order[0], 
                         size=s_order[1] if s_order[1] != -1 else self.getposition(s_order[0]).size, 
                         name=s_order[0]._name)
        
        #buy_list = sorted(buy_list, key=lambda x: x[2], reverse=False)
        random.shuffle(buy_list)
        for b_order in buy_list:
            self.data_tracker = b_order[0]
            self.log('买单创建, 代码: %s, 价格: %.2f, 股数: %i, 现有持仓: %i' % 
                        (b_order[0]._name, 
                        self.inds[b_order[0]]['dataclose'][0], 
                        b_order[1],
                        self.getposition(b_order[0]).size))
            self.order = self.buy(data=b_order[0], size=b_order[1], name=b_order[0]._name)


    #def stop(self):
        #for d in self.datas:
        #    if self.getposition(d).size > 0:
        #        self.profit_i[d._name] += self.getposition(d).size * d.close[0]
        #self.log('Ending Value:%.2f.' %
        #        (self.cerebro.broker.getvalue()), doprint=False)
        #print(sorted(self.profit_i.items(), key=lambda x: x[1]))
        #print(sum([x[1] for x in self.profit_i.items()]))
        #for d in self.datas:
        #    if self.getposition(d).size > 0:
        #        self.profit_i[d._name] += self.getposition(d).size * self.inds[d]['dataclose'][0]
        #print(sorted(self.profit_i.items(), key= lambda x:x[1]))
        #return self.profit_i

# 单次回测

In [4]:
date_idx = pd.read_csv('.\\Data\\2019\\000001.SZ.csv', converters={'trade_date':lambda x:pd.to_datetime(x)}).trade_date


if __name__ ==  '__main__':
    
    # Create a cerebro entity
    cerebro = bt.Cerebro(tradehistory=True)

    # Add a strategy
    strats = cerebro.addstrategy(BW_comp, printlog=False, units=350)

    s_date = datetime.date(2020,12,31) - datetime.timedelta(days=385)
    e_date = datetime.date(2020,12,31)

    
    stock_index = '399300.SZ'
    pro = ts.pro_api('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6')
    index_list = np.unique(pro.index_weight(index_code=stock_index,
                                            start_date=s_date.strftime('%Y%m%d'),
                                            end_date=e_date.strftime('%Y%m%d')).con_code).tolist()

    for s_code in index_list:
        df = pd.read_csv('.\\Data\\2019\\' + s_code + '.csv', converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
        #df = pd.read_csv('.\\Data\\current\\' + s_code + '.csv', converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
        df_new = pd.DataFrame([[x,0,0,0,0,0,0] for x in date_idx if x not in df.index], columns=['trade_date', 'open', 'high', 'low', 'close', 'volume', 'openinterest'])
        df_new.set_index('trade_date', inplace=True)
        if len(df_new) != 0:
            df = pd.concat([df_new, df])
        df.sort_index(inplace=True)
        data = bt.feeds.PandasData(dataname=df,fromdate=s_date,todate=e_date)

        # Add the index Data Feed to Cerebo
        cerebro.adddata(data, name=s_code)

    # Set cash inside the strategy
    cerebro.broker = bt.brokers.BackBroker(coc=True)
    cerebro.broker.setcash(50000000)

    # Set commission
    #cerebro.broker.setcommission()

    # Print out the starting conditions
    start_value = cerebro.broker.getvalue()
    #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    final_result = cerebro.run(maxcpus=16)

    # Print out the final result
    final_value = cerebro.broker.getvalue()
    #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))
    #profit.append((final_value - start_value) / start_value * 100)

    #cerebro.plot(iplot=False)

[('688012.SH', -72867.0), ('002153.SZ', -54143.0), ('600867.SH', -39871.0), ('600583.SH', -39777.0), ('002010.SZ', -39473.0), ('601696.SH', -38914.0), ('688008.SH', -38549.0), ('000703.SZ', -38349.0), ('002008.SZ', -35943.0), ('002371.SZ', -34410.0), ('600390.SH', -34222.00000000003), ('601377.SH', -33060.0), ('600584.SH', -32772.0), ('600104.SH', -32365.0), ('300498.SZ', -31320.0), ('600233.SH', -30040.0), ('000938.SZ', -28739.999999999985), ('600837.SH', -26939.0), ('002236.SZ', -26236.0), ('600498.SH', -25500.0), ('300529.SZ', -24797.000000000015), ('600153.SH', -24692.0), ('601816.SH', -24591.0), ('002463.SZ', -24293.0), ('600161.SH', -24028.0), ('601100.SH', -23820.99999999997), ('600221.SH', -23653.0), ('601138.SH', -22348.0), ('601618.SH', -22229.0), ('601989.SH', -22162.0), ('002714.SZ', -21771.999999999985), ('603195.SH', -21699.000000000015), ('603019.SH', -21567.0), ('601198.SH', -20736.0), ('600111.SH', -20228.0), ('600369.SH', -20108.0), ('600050.SH', -19818.0), ('603156.S

# 循环调参

In [5]:
profit_dict = {}

# 904ff4752522814dca00e032a709fdfc26d8744913500ef204e02157
# 74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6

date_idx = pd.read_csv('.\\Data\\2019\\000001.SZ.csv', converters={'trade_date':lambda x:pd.to_datetime(x)}).trade_date

def getdata_mul(s_code, start=datetime.date(2020, 12, 31)-datetime.timedelta(days = 385),
                end = datetime.date(2020, 12, 31)):
    global cerebro, date_idx
    df = pd.read_csv('.\\Data\\2019\\'+s_code+'.csv', 
                     converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
    df_new = pd.DataFrame([[x,0,0,0,0,0,0] for x in date_idx if x not in df.index], columns=['trade_date', 'open', 'high', 'low', 'close', 'volume', 'openinterest'])
    df_new.set_index('trade_date', inplace=True)
    if len(df_new) != 0:
        df = pd.concat([df_new, df])
    df.sort_index(inplace=True)
    data = bt.feeds.PandasData(dataname=df, fromdate=s_date, todate=e_date)
    cerebro.adddata(data, name=s_code)
    return

s_date = datetime.date(2020, 12, 31) - datetime.timedelta(days = 385)
e_date = datetime.date(2020, 12, 31)


#stock_index = '399300.SZ'
#pro = ts.pro_api('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6')
#index_list = np.unique(pro.index_weight(index_code=stock_index,
#                                        start_date=s_date.strftime('%Y%m%d'),
#                                        end_date=e_date.strftime('%Y%m%d')).con_code).tolist()


if __name__ == '__main__':
    for i in np.arange(1.5,3,0.1):
        # Create a cerebro entity
        cerebro = bt.Cerebro(tradehistory=True)

        # Add a strategy
        strats = cerebro.addstrategy(BW_comp, printlog=False, units=len(index_list), wave_period_buy=3, wave_period_sell=6, bollinger_dev = float(i))
        t_list = [utilsJ.MyThread(getdata_mul, (s_code, s_date, e_date)) for s_code in index_list]
        [t.start() for t in t_list]
        [t.join() for t in t_list]
        #for s_code in index_list:
        #    df = pd.read_csv('.\\Data\\2020\\'+s_code+'.csv',
        #                     converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
        #    data = bt.feeds.PandasData(dataname=df, fromdate=s_date, todate=e_date)

            # Add the index Data Feed to Cerebo
        #    cerebro.adddata(data, name=s_code)
        
        # Set cash inside the strategy
        cerebro.broker = bt.brokers.BackBroker(coc=True)
        cerebro.broker.setcash(5000 * len(index_list) * 100)

        # Set commission
        #cerebro.broker.setcommission()

        # Print out the starting conditions
        start_value = cerebro.broker.getvalue()
        #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

        # Run over everything
        cerebro.run(maxcpus=16)

        # Print out the final result
        final_value = cerebro.broker.getvalue()
        #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
        #print('(%s, %s) Net Profit: %.2f%%' % (s[0], s[1], (final_value-start_value)/start_value*100))
        print((i, (final_value - start_value) / start_value * 100))

(1.5, 5.246531988472623)
(1.6, 5.682348703170029)
(1.7000000000000002, 5.0715475504322765)
(1.8000000000000003, 5.0052449567723345)
(1.9000000000000004, 5.0324783861671465)
(2.0000000000000004, 5.0113953890489915)
(2.1000000000000005, 5.0845659942363115)
(2.2000000000000006, 4.579751585014409)
(2.3000000000000007, 4.249139481268012)
(2.400000000000001, 4.145895677233429)
(2.500000000000001, 4.35494121037464)
(2.600000000000001, 3.957588472622479)
(2.700000000000001, 4.0924916426512965)
(2.800000000000001, 4.134165994236311)
(2.9000000000000012, 3.7177965417867433)
